In [11]:
from tkinter import *
import tkinter as tk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import csv
import sqlite3
import pandas as pd
import re
from sklearn.feature_extraction import text
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
from nltk.stem import WordNetLemmatizer 
import tkinter.scrolledtext as tkscrolled
  

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


# Global Varaiables
dict1 ={}
TopNr = 10
data = []
posindex = []
#td-idf term-document matrix
matrix = []
#term list from td-idf term-document matrix
tml = []
#hotol name with index
namelist = []
#the hyperlink of result
link = " "
m = []

#Read info from csv files and compute tf-idf matrix from dataset
def compute_dataset():
    global matrix
    global namelist
    global tml
    global m
    csv_file=open('pr_result.csv') 
    csv_reader_lines = csv.reader(csv_file)
    next(csv_reader_lines)
    csv.field_size_limit(100000000)
    for one_line in csv_reader_lines:
        terms = one_line[1]
        hotel = one_line[0]
        m.append(terms)
        namelist.append(hotel)
        #del one_line[-1]
    csv_file.close()
    vectorizer = TfidfVectorizer(use_idf = TRUE, stop_words="english")
    matrix = vectorizer.fit_transform(m)
    tml = vectorizer.get_feature_names()
    read_file('total_info.csv')
    print("Tf-idf matrix compute finished.")
    

# get top words, otherHotels, and median review score for each cluster
def getClusterInfo(hotelName, city, clusterGroup, k):
    clusterGroup = int(clusterGroup)
    
    #### reading initial dataset ######
    
    #load group_hotel
    group_hotel = pd.read_pickle("./group_hotel.pkl")
    
      
    ##### filter the dataframe based on city #####

    #filter dataframe by hotel city .reset_index(drop=True) is optional
    cityDataframe = group_hotel[group_hotel['Hotel_City']==city].reset_index(drop=True)


    ###### get models used for kmeans ######
    
    #add more stop words based on the dataset
    useless_words = text.ENGLISH_STOP_WORDS.union(["room","rooms","run","st","couldn","location","wasn","hotel","stayed","place","best","positive","negative","time","big","minutes","did","day","didn","near","area","staff","city","close","free","service","really","like","stay","bed","beds","bit","es","just","little","night"])

    # for each hotel, the 'all_reviews' cell is changed to a row of terms corresponding to all terms in all rows of 'all_reviews'
    vectorizer = TfidfVectorizer(stop_words=useless_words, use_idf=True, smooth_idf=True) #create vectorizer function which make useful words into tokens
    tfidfMatrix = vectorizer.fit_transform(cityDataframe['all_reviews'])   #create a (hotel,term) matrix

       
    #'all_reviews' are clustered based on the terms contained
    model = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=1)
    model.fit(tfidfMatrix)
    
    
    clusters = model.labels_.tolist()
    cityDataframe['Clusters'] = clusters
    
        
    ######## get top 20 words #######
    order_centroids = model.cluster_centers_.argsort()[:, ::-1] 

    terms = vectorizer.get_feature_names()

    
    topWords = []
    for index in order_centroids[clusterGroup, :20]:   #for each value(index) of this cluster's first 2o values
        topWords.append(' %s' % terms[index])   
        
            
    ##### other hotels in same cluster ######
    if city == 'Amsterdam':
        totalClusters = 12
    elif city == 'Barcelona':
        totalClusters = 14
    elif city == 'London':
        totalClusters = 11
    elif city == 'Milan':
        totalClusters = 16
    elif city == 'Paris':
        totalClusters = 13
    elif city == 'Vienna':
        totalClusters = 13
        
    loadedDF = pd.read_pickle("./%s_%dClusters.pkl" %(city, totalClusters))
    otherHotels = loadedDF[clusterGroup].loc[loadedDF[clusterGroup] != hotelName].loc[loadedDF[clusterGroup].notnull()].reset_index(drop=True)
    
       
    #### median hotel rating score of this cluster ####
    medianScore = cityDataframe['Average_Score'].groupby(cityDataframe['Clusters']).median().loc[clusterGroup]
    
        
    return topWords, otherHotels, medianScore    #(list, pandas series, numpy float)
    

    
#read csv file
def read_file(name):
    global data
    csv_file=open(name) 
    csv_reader_lines = csv.reader(csv_file)
    next(csv_reader_lines)
    csv.field_size_limit(100000000)
    for one_line in csv_reader_lines:
        data.append(one_line)
    csv_file.close()
    #print(data)

    
# Get input query from user
def handle_input(sender):
    listNodes.delete(0, END)
    query = sender.get()
    terms = []
    st = ""
    for i in range(0, len(query)):
        if query[i] != " ":
            st = st + str(query[i])
        else:
            #print(st)
            terms.append(st)
            st = ""
        if i == len(query)-1:
            terms.append(st)
    #print(terms)
    count = 0
    pos = []
    for token in terms:
        for term in tml:
            if token == term:
                pos.append(count)
                count = 0
            else:
                count = count + 1
            if count == len(tml) - 1:
                pos.append(-1)
                count = 0
    grade = []
    xlist = matrix.toarray()
    for one_pos in pos:
        if one_pos != -1:
            one_pos_grade = [row[one_pos] for row in xlist]
            grade.append(one_pos_grade)
    total_grade = np.sum(grade, axis=0)
    final = total_grade.argsort()[::-1]
    for i in namelist:
        for t in terms:
            if t == i:
                listNodes.insert(END, i)
    listRank.delete(0, END)
    if np.all(final==0):
        listRank.insert(END, "No ranking hotel found")
    else:
        for i in range(0,TopNr):
            index = final[i]
            listRank.insert(END, namelist[index])
    
    result_Boolean=boolean_search(terms)
    if len(result_Boolean)== 0 :
        listNodes.insert(END, 'Wrong input keyword, or nothing is input')
    else:
        listNodes.insert(END, 'Boolean IR search result')
        for w in result_Boolean:
            listNodes.insert(END,w[0])
        
    


# Search button click event    
def button_click():
    clusterResult.delete(0, END)
    handle_input(inp)


#show info when an item in the listbox is selected
# def onselect(event, listbox):
#     w = event.widget
#     try:
#         index = int(w.curselection()[0])    #index stores the index of the first thing from the listbox that was selected
#     except IndexError:
#         return
#     if listbox is listNodes:
#         print("get into event call 1")
#         value = w.get(index)    #get the name of selected hotel, and put it in a list
#         listSelection.delete(0, END)
#         for h in data:
#             if value == h[0]:
#                 listSelection.insert(END, "Address: " + h[1])
#                 listSelection.insert(END, "Average Score: " + h[3])
#                 listSelection.insert(END, "Total Reviews: " + h[5])
#                 listSelection.insert(END, "Total Positive Comments:" + h[6])
#                 listSelection.insert(END, "Total Negative Comments:" + h[4])
#         if len(listSelection.get(0, END)) == 0:                              #if there is no hotel listed in the listbox
#             listSelection.insert(END, "Error: Can not find data")
#     if listbox is listSelection:
#         #return on_second_box(idx, w.get(idx))
#         return
#     if listbox is listRank:
#         return
#     if listbox is listreview:
#         value = w.get(index)    #get the name of selected hotel, and put it in a list
#         listreview.delete(0, END)
#         i = 0
#         for h in namelist:
#             if value == h:
                
#                 listreview.insert(END, m[i])
        
#             i = i+1
       
def reSelect(event):
    w = event.widget
    try:
        index = int(w.curselection()[0])    #index stores the index of the first thing from the listbox that was selected
    except IndexError:
        return
    value = w.get(index)
    #print("positive review value:")
    #print(value)
    
    listSelection.delete(0, END)
    for h in data:
        if value == h[0]:
            listSelection.insert(END, "Address: " + h[1])
            listSelection.insert(END, "Average Score: " + h[3])
            listSelection.insert(END, "Total Reviews: " + h[5])
            listSelection.insert(END, "Total Positive Comments:" + h[6])
            listSelection.insert(END, "Total Negative Comments:" + h[4])
    if len(listSelection.get(0, END)) == 0:                              #if there is no hotel listed in the listbox
        listSelection.insert(END, "Error: Can not find data")
    
#     listreview.delete(0, END)
    listreview.delete("1.0", END)
    inc = 0
    for hn in namelist:
        if value == hn:
#             print("find the hotel in the list")
#             listreview.insert(END, m[inc])
            old = m[inc]
            new = old.split()[:2000]
            listreview.insert(tk.INSERT, new)
        inc = inc + 1
    
    
def curSelect(event):
    #find the listbox widget that is calling for this function
    caller = event.widget
    selectedIndex = caller.curselection()
    if selectedIndex:
        selectedName = caller.get(selectedIndex[0])    #select hotel name stored
    
    #find city of the hotel using group_hotel dataframe
    group_hotel = pd.read_pickle('group_hotel.pkl')
    city = group_hotel[group_hotel['Hotel_Name']==selectedName]['Hotel_City'].to_string(index=False).strip()   #return the city in string
    
    #load the pkl of the city to find clusterGroup
    if city == 'Amsterdam':
        totalClusters = 12
    elif city == 'Barcelona':
        totalClusters = 14
    elif city == 'London':
        totalClusters = 11
    elif city == 'Milan':
        totalClusters = 16
    elif city == 'Paris':
        totalClusters = 13
    elif city == 'Vienna':
        totalClusters = 13
    else:
        print(f"city {city} is not clustered")
        
    cityPkl = pd.read_pickle("./%s_%dClusters.pkl" %(city, totalClusters))
    clusterGroup = cityPkl[cityPkl==selectedName].stack().to_string().split()[1]   #clusterGroup returned as string
    
        
    topWords, otherHotels, medianScore = getClusterInfo(selectedName, city, clusterGroup, totalClusters)
    
    clusterResult.delete(0, END)
    
    clusterResult.insert(END, 'Cluster Information:\n')
    clusterResult.insert(END, "\n")
    clusterResult.insert(END, "Selected Hotel: " + selectedName + "\n")
    clusterResult.insert(END, "\n")
    clusterResult.insert(END, "Hotel City: " + city + "\n")
    clusterResult.insert(END, "\n")
    clusterResult.insert(END, "Hotel Cluster Group: " + clusterGroup + "\n")
    clusterResult.insert(END, "\n")
    clusterResult.insert(END, "The Median Review Score for this Cluster Group is: " + str(medianScore) +"\n")
    clusterResult.insert(END, "\n")
    clusterResult.insert(END, "Top 20 Words of This Cluster:\n")
    for word in topWords:
        clusterResult.insert(END, word + "\n")
    clusterResult.insert(END, "\n")
    clusterResult.insert(END, "Hotels in the Same Cluster Group:\n")
    for i in otherHotels:
        clusterResult.insert(END, i + "\n")

def create_dictionary(file_name):
    # raw documents to tf-idf matrix:
    
    global dict1

    csv.field_size_limit(100000000)
    csv_file=open(file_name)    
    csv_reader_lines = csv.reader(csv_file)   
    date=[]    
    renshu = 0
    for one_line in csv_reader_lines:
        date.append(one_line)    
        renshu = renshu + 1  
    #print(len(date))
    #print(renshu)
    
    pre = []
    #print(date[1493][0])
    #print(date[1493][1])
    stop_words = set(stopwords.words('english')) 
    for i in range(1,len(date)):
        name = date[i][0]
        review = date[i][1]
        word_tokens = word_tokenize(review) 
        #filtered_sentence = [w for w in word_tokens if not w in stop_words] 
        filtered_sentence = [] 
        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w) 
        f_review = [name, filtered_sentence]
        pre.append(f_review)
        
    lemmatizer = WordNetLemmatizer() 
    lemmatizer.lemmatize("rocks")
    pre4 = pre
    for i in range (0,len(pre)):
        data = pre4[i][1]
        data2=data
        for j in range(0,len(data)):
            data3 = data[j]
            data_lem = lemmatizer.lemmatize(data3)
            data2[j] = data_lem
        pre4[i][1] = data2
    

    
    lancaster_stemmer = LancasterStemmer()
    pre2 = pre4
    for i in range (0,len(pre4)):
        dat = pre2[i][1]
        dat2=dat
        for j in range(0,len(dat)):
            dat3 = dat[j]
            data_stem = lancaster_stemmer.stem(dat3)
            dat2[j] = data_stem
        pre2[i][1] = dat2
    
    
    #print("This step is finish")
    a = "and"
    for i in range (0, len(pre2)):
    
        li = pre2[i][1]
        #print(li)
        name = pre2[i][0]
        #print(name)
        for w in li:
            if w != a:
                if w not in dict1:
                    #print("add new word")
                    dict2={}
                    dict2[name] = 1
                    dict1[w] = dict2
                    #rint(dict1[w])
                else:
                    p= dict1[w]
                    if name not in p:
                        p[name] = 1
                        dict1[w] = p
                        #int(dict1[w])
                    else:
                        f = p[name]
                        p[name] = f+1
                        dict1[w] = p 
                        #rint(dict1[w])

    print("prepare step is finish")
  
        

   # with open('pf_result.csv', 'w', newline='') as csvfile:
    #    writer = csv.writer(csvfile)
      #  for row in pre2:
        #    writer.writerow(row)
def boolean_search(input_t):
    input_txt = []
    stop_words = set(stopwords.words('english')) 
    lemmatizer = WordNetLemmatizer() 
    lemmatizer.lemmatize("rocks")
    lancaster_stemmer = LancasterStemmer()
    
    for w in input_t: 
        if w not in stop_words: 
            lem = lemmatizer.lemmatize(w)
            stem = lancaster_stemmer.stem(lem)
            input_txt.append(stem) 
    #print (input_txt)
    
    
    AND = "and"
    OR = "or"
    #NOT = "not"
    case = 5
    for i in input_txt:
        if i in dict1.keys():
            case = 0
    if case == 0:
        if AND in input_txt:
            case = 1
            #print("and")
            process = {}
            a = 0 
            for i in input_txt:
                if i in dict1.keys():
                    l = dict1[i]
                    process[a] = l
                    a = a+1
        elif OR in input_txt:
            #print("or")
            case = 2
            process = {}
            a = 0 
            for i in input_txt:
                if i in dict1.keys():
                    l = dict1[i]
                    process[a] = l
                    a = a+1
        else:
            #print("nothing")
            case = 3
            process = {}
            a = 0 
            for i in input_txt:
                if i in dict1.keys():
                    l = dict1[i]
                    process[a] = l
                    a = a+1
   # if OR in input_txt:
   # if NOT in input_txt:
    if case == 5:
        #print(case)
        #print("print empty input")
        pr = {}
        
        
    if case == 1 or case == 3:
        #print(case)
        #print("case and or nothing")
        pr = process[0]
        for z in range(1,len(process)):
            a = process[z]
            for i,j in a.items():
                if i in pr.keys():
                    
                    pr[i]= pr[i]+j
                else:
                    pr[i]=j
    if case == 2:
        #print("case or")
        pr = process[0]
        for z in range(1,len(process)):
            a = process[z]
            for i,j in a.items():
                if i in pr.keys():
                    if pr[i] <= j:
                        pr[i]=j
                else:
                    pr[i]=j
        
    search_final= sorted(pr.items(), reverse=True,key=lambda d: d[1]) 
    return search_final
                    




# Search page GUI
searchpage = Tk()
searchpage.title("Hotel Search")
# searchpage.geometry("680x300")
lab1 = Label(searchpage, text = "IR Project Search Engine")
# lab.pack()
lab1.grid(row=0, column=1)
# lab.config(justify = CENTER)
lab2 = Label(searchpage, text = "Result List")
lab2.grid(row=2, column=0)
lab3 = Label(searchpage, text = "Ranking List")
lab3.grid(row=2, column=2)
lab4 = Label(searchpage, text = "Recommendation Info")
lab4.grid(row=5, column=1)
lab5 = Label(searchpage, text = "Review Keywords")
lab5.grid(row=7, column=1)
inp = Entry(searchpage, width = 30)
# inp.pack()
inp.grid(row=1, column=1)
# lab.config(justify = CENTER)
btn = Button(searchpage, text = "Search")
# btn.pack()
btn.grid(row=2, column=1)
btn.config(command = button_click)
#lab2 = Label(searchpage, text = "Result")
#lab.grid(row=3, column=0)


frm = Frame(searchpage)
frm2 = Frame(searchpage)
frm3 = Frame(searchpage)
frm4 = Frame(searchpage)
frm.grid(row=4, column=0, rowspan=2, sticky=N+S)
frm2.grid(row=4, column=1, sticky=E+W+N) # row=4
frm3.grid(row=6, column=1) # row=5
frm4.grid(row=8,column=1) # row=6
# searchpage.rowconfigure(1, weight=1)
# searchpage.columnconfigure(1, weight=1)
scrollbar = Scrollbar(frm, orient="vertical")
scrollbar.pack(side=RIGHT, fill=Y)
scrollbar2 = Scrollbar(frm2, orient="horizontal")
scrollbar2.pack(side=BOTTOM, fill=X)
scrollbar3 = Scrollbar(frm3, orient="vertical")
scrollbar3.pack(side=RIGHT, fill=Y)
# scrollbar4 = Scrollbar(frm4, orient="vertical")
# scrollbar4.pack(side=RIGHT, fill=Y)

listNodes = Listbox(frm, width=60, yscrollcommand=scrollbar.set)
listNodes.pack(expand=True, fill=Y)
# listNodes.bind('<<ListboxSelect>>',lambda e:onselect(e, listNodes))
listNodes.bind('<<ListboxSelect>>',lambda event: reSelect(event))

scrollbar.config(command=listNodes.yview)

listSelection = Listbox(frm2, width=60, height=5, xscrollcommand=scrollbar2.set)
listSelection.pack(expand=True, fill=X)
# listSelection.bind('<<ListboxSelect>>',lambda e:onselect(e, listSelection))

scrollbar2.config(command=listSelection.xview)

listRank = Listbox(searchpage, width=60)
listRank.grid(row=4, column=2, rowspan=2, sticky=N+S)
# listRank.bind('<<ListboxSelect>>',lambda e:onselect(e, listRank))
listRank.bind('<<ListboxSelect>>', lambda event: curSelect(event))


clusterResult = Listbox(frm3, width=60, yscrollcommand=scrollbar3.set)
clusterResult.pack(expand=True, fill=Y)
# clusterResult.bind('<<ListboxSelect>>',lambda e:onselect2(e, clusterResult))


scrollbar3.config(command=clusterResult.yview)

# listreview = Listbox(frm4, width=60, yscrollcommand=scrollbar4.set)
# listreview.pack(expand=True, fill=Y)

# scrollbar4.config(command=listreview.yview)
listreview = tkscrolled.ScrolledText(frm4, wrap = tk.WORD, width = 60, height = 15)
listreview.pack(expand=True, fill='both')


compute_dataset()
create_dictionary('prr_result.csv')
searchpage.mainloop()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Paws\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Paws\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Paws\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Tf-idf matrix compute finished.
prepare step is finish
